In [ ]:
from transformers import AutoTokenizer,TFAutoModel
tokenizer = AutoTokenizer.from_pretrained("../input/huggingface-roberta-variants/roberta-base/roberta-base",max_length=256)

In [ ]:
import pandas as pd
path=[
    "../input/commonlitreadabilityprize/sample_submission.csv",
    "../input/commonlitreadabilityprize/test.csv",
    "../input/commonlitreadabilityprize/train.csv"
]

df_train = pd.read_csv(path[2])
df_test = pd.read_csv(path[1])
df_ss = pd.read_csv(path[0])

In [ ]:
df_train = df_train.drop(['url_legal','license'],axis='columns')
df_test = df_test.drop(['url_legal','license'],axis='columns')

In [ ]:
X= df_train['excerpt']
y=df_train['target'].values

X_test = df_test['excerpt']

In [ ]:
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf

n_splits = 5
df_train['bin'] = pd.cut(df_train.target,20, labels=[i for i in range(20)])
skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
gen_skf = skf.split(df_train.id, y=df_train.bin)


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import numpy as np
import os
SEEDS = [42]
cv_pred = np.zeros((len(X),))
test_pred = np.zeros((len(X_test)))
for seed in SEEDS:
    #skf = StratifiedKFold(n_splits=n_splits,random_state=seed,shuffle=True)
    #gen_skf = skf.split(df_train.id, y=df_train.bin)
    kfold = KFold(n_splits=5, shuffle= True , random_state= seed)
    for fold, (idx_train, idx_val) in enumerate(kfold.split(X,y)):
        X_train = X[idx_train]
        X_labels = y[idx_train]
        X_val = X[idx_val]
        val_labels = y[idx_val] 
        test_labels = np.zeros((len(X_test),))
        
        #training_features = tokenizer(list(X_train),padding="max_length", truncation=True,return_tensors='tf')
        val_features = tokenizer(list(X_val),padding="max_length",max_length=256,truncation=True,return_tensors='tf')
        testing_features = tokenizer(list(X_test),padding="max_length",max_length=256,truncation=True,return_tensors='tf')   
        #training_features = {x:training_features[x] for x in tokenizer.model_input_names}
        val_features = {x:val_features[x] for x in tokenizer.model_input_names}
   
        testing_features =  {x:testing_features[x] for x in tokenizer.model_input_names}
        #training_set = tf.data.Dataset.from_tensor_slices((training_features, X_labels))
        #training_set = training_set.batch(8)
        val_set = tf.data.Dataset.from_tensor_slices((val_features,val_labels))
        val_set = val_set.batch(8)
        test_set = tf.data.Dataset.from_tensor_slices((testing_features,test_labels))
        test_set = test_set.batch(8) 
        
    
        transformer_model = TFAutoModel.from_pretrained("../input/huggingface-roberta-variants/roberta-base/roberta-base",output_hidden_states=True)    
        input_ids = tf.keras.Input(shape=(256, ),dtype='int32')
        attention_mask = tf.keras.Input(shape=(256, ),dtype='int32')
        transformer = transformer_model({'input_ids':input_ids,'attention_mask':attention_mask})    
        hidden_states = transformer[0] # get output_hidden_states
        
        x = tf.keras.layers.Dense(128, activation='relu')(hidden_states[:,-1,:])
        x = tf.keras.layers.Dropout(0.025)(x)
        output = tf.keras.layers.Dense(1)(x)
        model = tf.keras.models.Model(inputs = [{'input_ids':input_ids,'attention_mask':attention_mask}],outputs = output)
        checkpoint_filepath = f'../input/fork-of-finetune-roberta-model-with-5-fold/model{seed}_{fold}'

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=tf.keras.losses.MeanSquaredError())
        
        model.load_weights(checkpoint_filepath)    
        cv_pred[idx_val] += model.predict(val_set)[:,0]/(len(SEEDS))
        print(mean_squared_error(cv_pred[idx_val],y[idx_val]))
        test_pred += model.predict(test_set)[:,0]/(n_splits*len(SEEDS))
    print(f'OOF{seed}=',mean_squared_error(cv_pred,y))

In [ ]:
submission = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")
submission.target = test_pred
submission.to_csv("submission.csv", index=False)